<a href="https://colab.research.google.com/github/PriyankaGona/MachineLearning-Assignments/blob/master/nlp_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

import warnings 
warnings.filterwarnings(action = 'ignore') 

import gensim 
from gensim.models import Word2Vec 
import string
import numpy as np
from keras.layers import Dense, Input, LSTM,GRU, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

import re

!pip install pyspellchecker
from spellchecker import SpellChecker

!pip install symspellpy
from symspellpy.symspellpy import SymSpell
import os
!wget https://github.com/mammothb/symspellpy/blob/master/symspellpy/frequency_dictionary_en_82_765.txt?raw=true \
  -O freq_dict.txt
  
import sys
import time


paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Using TensorFlow backend.


    100% |████████████████████████████████| 1.9MB 16.9MB/s 
--2019-04-23 01:02:54--  https://github.com/mammothb/symspellpy/blob/master/symspellpy/frequency_dictionary_en_82_765.txt?raw=true
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/mammothb/symspellpy/raw/master/symspellpy/frequency_dictionary_en_82_765.txt [following]
--2019-04-23 01:02:54--  https://github.com/mammothb/symspellpy/raw/master/symspellpy/frequency_dictionary_en_82_765.txt
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mammothb/symspellpy/master/symspellpy/frequency_dictionary_en_82_765.txt [following]
--2019-04-23 01:02:54--  https://raw.githubusercontent.com/mammothb/symspellpy/master/symspellpy/frequency_dictionary_en_82_765.txt
Resolving raw.githubus

In [0]:
TRAIN_DATA_FILE = 'train.csv'
TEST_DATA_FILE = 'test.csv'

train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_train = train_df[list_classes].values
y_test = test_df[list_classes].values

sentences = train_df["comment_text"].fillna("").values

In [0]:
filter_words = set(stopwords.words('english')) 

data = [re.sub(r'[^a-zA-Z ]', '', sentence).lower() for sentence in sentences]
data_words = [sentence.split() for sentence in data]
# Remove stopwords
#for row in range(len(data)):
  #data[row] = [w for w in data[row] if not w in filter_words]

In [0]:
data_spell = [' '.join(sentence) for sentence in data_words]

data_words

[['cocksucker', 'before', 'you', 'piss', 'around', 'on', 'my', 'work'],
 ['hey',
  'what',
  'is',
  'it',
  'talk',
  'what',
  'is',
  'it',
  'an',
  'exclusive',
  'group',
  'of',
  'some',
  'wp',
  'talibanswho',
  'are',
  'good',
  'at',
  'destroying',
  'selfappointed',
  'purist',
  'who',
  'gang',
  'up',
  'any',
  'one',
  'who',
  'asks',
  'them',
  'questions',
  'abt',
  'their',
  'antisocial',
  'and',
  'destructive',
  'noncontribution',
  'at',
  'wpask',
  'sityush',
  'to',
  'clean',
  'up',
  'his',
  'behavior',
  'than',
  'issue',
  'me',
  'nonsensical',
  'warnings'],
 ['bye',
  'dont',
  'look',
  'come',
  'or',
  'think',
  'of',
  'comming',
  'back',
  'tosser'],
 ['you',
  'are',
  'gay',
  'or',
  'antisemmitian',
  'archangel',
  'white',
  'tigermeow',
  'greetingshhhuh',
  'there',
  'are',
  'two',
  'ways',
  'why',
  'you',
  'do',
  'erased',
  'my',
  'comment',
  'about',
  'ww',
  'that',
  'holocaust',
  'was',
  'brutally',
  'slayin

In [0]:
'''

spell = SpellChecker()
start = time.time()
data_words_1 = [[spell.correction(word) if len(word) < 10 else word for word in sentence ] for sentence in data_words]
end = time.time()


from nltk.stem import WordNetLemmatizer 
  
lemmatizer = WordNetLemmatizer() 

print("\nProcessing finished in %f minutes" % ((end - start)/60))'''

'\n\nspell = SpellChecker()\nstart = time.time()\ndata_words_1 = [[spell.correction(word) if len(word) < 10 else word for word in sentence ] for sentence in data_words]\nend = time.time()\n\n\nfrom nltk.stem import WordNetLemmatizer \n  \nlemmatizer = WordNetLemmatizer() \n\nprint("\nProcessing finished in %f minutes" % ((end - start)/60))'

In [0]:
# maximum edit distance per dictionary precalculation
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
# load dictionary
term_index = 0  # column of the term in the dictionary text file
count_index = 1  # column of the term frequency in the dictionary text file
if not sym_spell.load_dictionary("freq_dict.txt", term_index, count_index):
  print("Dictionary file not found")

start = time.time()

data = [[suggestion.term for suggestion in sym_spell.lookup_compound(sentence,
                                        2)] for sentence in data]

end = time.time()

print("\nProcessing finished in %f minutes" % ((end - start)/60))

data_words = [sentence.split() for sentence in data]

KeyboardInterrupt: ignored

In [0]:
data_words[3]

In [0]:
model1 = gensim.models.Word2Vec(data_words, min_count = 5,  
                              size = 100)
vectors = model1.wv

In [0]:
vectors.most_similar("stupid")

In [0]:
embed_layer = vectors.get_keras_embedding(train_embeddings=False)

In [0]:
# Convert the list of sentences to a list of number sequences,
# where each number is a unique identifier to a word.
seq_limit = 30

tokenizer = Tokenizer(num_words= 100000)
tokenizer.fit_on_texts(data)

data_seq = tokenizer.texts_to_sequences(data)
data_padded = pad_sequences(data_seq, maxlen=30)

In [0]:
index_dict = tokenizer.word_index
print("Total number of words: %d"% len(index_dict))
num_words = min(100000, len(index_dict))+1
embed_matrix = np.zeros((num_words, 100))
num_wrong = []
for word, index in index_dict.items():
    if word in vectors:
      embed_matrix[index] = vectors[word]
    else:
      num_wrong.append(word)
print('Null word embeddings: %d' % np.sum(np.sum(embed_matrix, axis=1) == 0))

embed_layer_2 = Embedding(embed_matrix.shape[0],
                            embed_matrix.shape[1],
                            weights=[embed_matrix],
                            input_length=seq_limit,
                            trainable=False)
print(num_wrong)

In [0]:
'cannot' in vectors

In [0]:
'cannot' in data_words

In [0]:
model = Sequential()
model.add(embed_layer)

input_array = [[0]]

model.compile('rmsprop', 'mse')
print(model.summary())
output_array = model.predict(input_array)

In [0]:
output_array

In [0]:
vectors['the']

In [0]:
gru_model = Sequential()
gru_model.add(embed_layer)
gru_model.add(GRU(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))
gru_model.add(Dense(6, activation="sigmoid"))
gru_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])'''

In [0]:
#gru_model.fit(x_train, y_train, batch_size=256, epochs=2, validation_split=0.1, verbose=2);